# Cox time-varying proportional hazards model with hyperparameter tuning

In [1]:
import sys
import logging
from pathlib import Path

import pandas as pd
from prefect import Flow

from nba_survival.model.tasks import (
    SurvivalData,
    SegmentData,
    InitializeLifelines,
    FitLifelinesModel,
    CollapseData,
    PredictLifelines,
    ConcordanceIndex,
    HyperparameterTuning
)
from nba_survival.model.tasks.meta import META

## Load the data

In [2]:
df = pd.concat(
    pd.read_csv(fpath, sep="|", dtype={"GAME_ID": str}, index_col=0)
    for fpath in Path("..", "nba-data", "2018-19", "model-data").glob("data_*.csv")
).reset_index(drop=True)

In [3]:
df[
    [META["id"]] + META["dynamic"] + META["static"]
].head(n=5)

,GAME_ID,SCOREMARGIN,HOME_LINEUP_PLUS_MINUS,VISITOR_LINEUP_PLUS_MINUS,HOME_NET_RATING,VISITOR_NET_RATING,HOME_W_PCT,VISITOR_W_PCT,LAST_GAME_WIN,HOME_GAMES_IN_LAST_3_DAYS,HOME_GAMES_IN_LAST_5_DAYS,HOME_GAMES_IN_LAST_7_DAYS,VISITOR_GAMES_IN_LAST_3_DAYS,VISITOR_GAMES_IN_LAST_5_DAYS,VISITOR_GAMES_IN_LAST_7_DAYS
0,0021800001,0,0.1,-0.1,5.1,3.3,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0
1,0021800001,0,0.1,-0.1,5.1,3.3,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0
2,0021800001,0,0.1,-0.1,5.1,3.3,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0
3,0021800001,0,0.1,-0.1,5.1,3.3,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0
4,0021800001,0,0.1,-0.1,5.1,3.3,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0


## Build the model with hyperparameter tuning

In [4]:
format_data = SurvivalData(name="Convert input data to range form")
segdata = SegmentData(name="Create train and test data")
tuning = HyperparameterTuning(name="Hyperparameter tuning")
model = InitializeLifelines(name="Initialize Cox model")
trained = FitLifelinesModel(name="Fit Cox model")
collapsed_tune = CollapseData(name="Collapse tune data")
collapsed_test = CollapseData(name="Collapse test data")
predict = PredictLifelines(name="Predict partial hazard")
concord = ConcordanceIndex(name="Calculate C-Index")

In [5]:
with Flow(name="My model pipeline") as flow:
    alldata = format_data(df)
    data = segdata(alldata, splits=[0.8, 0.1], keys=["train", "tune", "test"])
    tune = collapsed_tune(data["tune"])
    params = tuning(data["train"], tune, max_evals=50)
    model_obj = model(params["best"])
    fitted = trained(model=model_obj, data=data["train"])
    testing = collapsed_test(data["test"])
    predt = predict(model=fitted, data=testing)
    cind = concord(data=testing, predt=predt)

In [6]:
output = flow.run()

[2021-02-13 02:54:57+0000] INFO - prefect.FlowRunner | Beginning Flow run for 'My model pipeline'
[2021-02-13 02:54:57+0000] INFO - prefect.TaskRunner | Task 'Convert input data to range form': Starting task run...


/usr/local/lib/python3.8/site-packages/lifelines/utils/__init__.py:1452: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv[duration_col] += delay


[2021-02-13 02:55:23+0000] INFO - prefect.TaskRunner | Task 'Convert input data to range form': Finished task run for task with final state: 'Success'
[2021-02-13 02:55:23+0000] INFO - prefect.TaskRunner | Task 'Create train and test data': Starting task run...
[2021-02-13 02:55:23+0000] INFO - prefect.Create train and test data | Setting the seed to 42
[2021-02-13 02:55:23+0000] INFO - prefect.Create train and test data | Dataset ``train`` has 983 rows
[2021-02-13 02:55:23+0000] INFO - prefect.Create train and test data | Dataset ``tune`` has 123 rows
[2021-02-13 02:55:23+0000] INFO - prefect.Create train and test data | Dataset ``test`` has 123 rows
[2021-02-13 02:55:23+0000] INFO - prefect.TaskRunner | Task 'Create train and test data': Finished task run for task with final state: 'Success'
[2021-02-13 02:55:24+0000] INFO - prefect.TaskRunner | Task 'GetItem': Starting task run...
[2021-02-13 02:55:24+0000] INFO - prefect.TaskRunner | Task 'GetItem': Finished task run for task with 

/usr/local/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()



Iteration 1: norm_delta = 0.01514, step_size = 0.90000, ll = -545584.55718, newton_decrement = 6.65872, seconds_since_start = 0.4
Iteration 2: norm_delta = 0.00228, step_size = 0.90000, ll = -364947.01335, newton_decrement = 0.20063, seconds_since_start = 0.7
Iteration 3: norm_delta = 0.00333, step_size = 0.90000, ll = -244526.92986, newton_decrement = 0.58946, seconds_since_start = 0.9
Iteration 4: norm_delta = 0.00268, step_size = 0.88200, ll = -164246.73108, newton_decrement = 0.55964, seconds_since_start = 1.2
Iteration 5: norm_delta = 0.00202, step_size = 0.86436, ll = -110726.61246, newton_decrement = 0.47504, seconds_since_start = 1.4
Iteration 6: norm_delta = 0.00173, step_size = 1.00000, ll = -75046.53681, newton_decrement = 0.38199, seconds_since_start = 1.8
Iteration 7: norm_delta = 0.00099, step_size = 1.00000, ll = -51259.70392, newton_decrement = 0.18459, seconds_since_start = 2.0
Iteration 8: norm_delta = 0.00069, step_size = 1.00000, ll = -35401.90172, newton_decrement 

...

Iteration 1: norm_delta = 0.69518, step_size = 0.90000, ll = -4080.54083, newton_decrement = 286.03111, seconds_since_start = 0.2
Iteration 2: norm_delta = 0.11269, step_size = 0.90000, ll = -3675.00517, newton_decrement = 3.84582, seconds_since_start = 0.4
Iteration 3: norm_delta = 0.00789, step_size = 0.90000, ll = -3586.75949, newton_decrement = 0.01481, seconds_since_start = 0.6
Iteration 4: norm_delta = 0.01541, step_size = 1.00000, ll = -3531.63284, newton_decrement = 0.03096, seconds_since_start = 0.8
Iteration 5: norm_delta = 0.01596, step_size = 0.98000, ll = -3495.54802, newton_decrement = 0.03702, seconds_since_start = 1.1
Iteration 6: norm_delta = 0.01661, step_size = 0.96040, ll = -3471.81956, newton_decrement = 0.05179, seconds_since_start = 1.6
Iteration 7: norm_delta = 0.01624, step_size = 0.94119, ll = -3456.20969, newton_decrement = 0.06768, seconds_since_start = 2.1
Iteration 8: norm_delta = 0.01522, step_size = 0.92237, ll = -3445.99912, newton_decrement = 0.08279, 

/usr/local/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()


Iteration 31: norm_delta = 0.00000, step_size = 0.98000, ll = -3555.15559, newton_decrement = 0.00001, seconds_since_start = 6.1Convergence completed after 31 iterations.


<lifelines.CoxTimeVaryingFitter: fitted with 113348 periods, 983 subjects, 586 events>
         event col = 'WIN'
         penalizer = 0.009713772398383114
number of subjects = 983
 number of periods = 113348
  number of events = 586
partial log-likelihood = -3555.16
  time fit was run = 2021-02-13 02:59:51 UTC

---
                               coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                            
HOME_GAMES_IN_LAST_3_DAYS     -0.01       0.99       0.06            -0.12             0.11                 0.89                 1.11
HOME_GAMES_IN_LAST_5_DAYS     -0.00       1.00       0.00            -0.01             0.01                 0.99                 1.01
HOME_GAMES_IN_LAST_7_DAYS     -0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
HOME_LINEUP_PLUS_MINUS         0.00       1.00       0.01            -0.01             0.02                 0.99                 1.02
HOME_NET_RATING                0.01       1.01       0.01             0.00             0.02                 1.00                 1.02
HOME_W_PCT                     0.14       1.15       0.14            -0.13             0.41                 0.88                 1.51
LAST_GAME_WIN                  0.00       1.00       0.00            -0.01             0.01                 0.99                 1.01
SCOREMARGIN                    0.03       1.03       0.00             0.03             0.04                 1.03                 1.04
VISITOR_GAMES_IN_LAST_3_DAYS   0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
VISITOR_GAMES_IN_LAST_5_DAYS  -0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
VISITOR_GAMES_IN_LAST_7_DAYS   0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
VISITOR_LINEUP_PLUS_MINUS     -0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
VISITOR_NET_RATING            -0.01       0.99       0.01            -0.02             0.00                 0.98                 1.00
VISITOR_W_PCT                 -0.05       0.95       0.15            -0.34             0.23                 0.71                 1.26

                                 z      p   -log2(p)
covariate                                           
HOME_GAMES_IN_LAST_3_DAYS    -0.13   0.89       0.16
HOME_GAMES_IN_LAST_5_DAYS    -0.01   0.99       0.01
HOME_GAMES_IN_LAST_7_DAYS    -0.01   1.00       0.01
HOME_LINEUP_PLUS_MINUS        0.08   0.94       0.09
HOME_NET_RATING               2.25   0.02       5.36
HOME_W_PCT                    1.02   0.31       1.71
LAST_GAME_WIN                 0.01   0.99       0.01
SCOREMARGIN                  15.07 <0.005     168.16
VISITOR_GAMES_IN_LAST_3_DAYS  0.00   1.00       0.00
VISITOR_GAMES_IN_LAST_5_DAYS -0.00   1.00       0.00
VISITOR_GAMES_IN_LAST_7_DAYS  0.01   1.00       0.01
VISITOR_LINEUP_PLUS_MINUS    -0.00   1.00       0.01
VISITOR_NET_RATING           -1.71   0.09       3.53
VISITOR_W_PCT                -0.36   0.72       0.48
---
Partial AIC = 7138.31
log-likelihood ratio test = 262.29 on 14 df
-log2(p) of ll-ratio test = 156.42

[2021-02-13 02:59:57+0000] INFO - prefect.TaskRunner | Task 'Fit Cox model': Finished task run for task with final state: 'Success'
[2021-02-13 02:59:57+0000] INFO - prefect.TaskRunner | Task 'Predict partial hazard': Starting task run...
[2021-02-13 02:59:57+0000] INFO - prefect.TaskRunner | Task 'Predict partial hazard': Finished task run for task with final state: 'Success'
[2021-02-13 02:59:57+0000] INFO - prefect.TaskRunner | Task 'Calculate C-Index': Starting task run...
[2021-02-13 02:59:57+0000] INFO - prefect.Calculate C-Index | Model has a C-index of 0.683
[2021-02-13 02:59:57+0000] INFO - prefect.TaskRunner | Task 'Calculate C-Index': Finished task run for task with final state: 'Success'
[2021-02-13 02:59:57+0000] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
